In [61]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.mam.*
import jline.util.matrix.*

In [62]:
val model = Network("model")

// Create nodes
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val source = Source(model, "Source")
val sink = Sink(model, "Sink")

// Create job class
val jobclass = OpenClass(model, "Class1", 0)

In [63]:
// Set service and arrival processes
delay.setService(jobclass, HyperExp(0.5, 3.0, 10.0))  // Hyperexponential distribution
queue1.setService(jobclass, Exp(1.0))  // Exponential service
source.setArrival(jobclass, Exp(0.1))  // Exponential arrivals

In [64]:
// Set routing matrix
val P = model.initRoutingMatrix()
P.set(jobclass, jobclass, Matrix("[0,1,0,0; 0,0,0,1; 1,0,0,0; 0,0,0,0]"))
model.link(P)

In [ ]:
// Create array of different solvers to compare - MVA first
val solvers = arrayOf(
    MVA(model),  // Moved MVA to first position
    CTMC(model, "cutoff", 10),
    FLD(model),
    MAM(model),
    NC(model),
    JMT(model, "seed", 23000, "verbose", false, "keep", true),
    SSA(model, "seed", 23000, "verbose", true, "samples", 10000)
)

// Execute each solver and display results
for (solver in solvers) {
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Solver failed: ${e.message}")
    }
}